In [1]:
import itertools
import numpy as np
from lattice2d_linear_env import Lattice2DLinearEnv

In [2]:
def value_iteration(env, theta=0.0001, discount_factor=1.0, max_iter = 10000):
    """
    Value Iteration Algorithm.
    
    Args:
        env: env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.
    """
    
    def one_step_lookahead(state, V):
        """
        Helper function to calculate the value for all action in a given state.
        
        Args:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.
        """
        A = np.zeros(env.nA)
        for a in range(env.nA):
            next_state, reward, done = env.P[state][a]
            A[a] += reward + discount_factor * V[next_state]
        return A
    
    V = np.zeros(env.nS)
    for i in range(max_iter):
        # Stopping condition
        delta = 0
        # Update each state...
        for s in range(env.nS - 3**(len(env.seq) - 2)):
            # Do a one-step lookahead to find the best action
            A = one_step_lookahead(s, V)
            best_action_value = np.max(A)
            # Calculate delta across all states seen so far
            delta = max(delta, np.abs(best_action_value - V[s]))
            # Update the value function. Ref: Sutton book eq. 4.10. 
            V[s] = best_action_value        
        # Check if we can stop 
        if delta < theta:
            break
    
    # Create a deterministic policy using the optimal value function
    policy = np.zeros([env.nS, env.nA])
    for s in range(env.nS):
        # One step lookahead to find the best action for this state
        A = one_step_lookahead(s, V)
        best_action = np.argmax(A)
        # Always take the best action
        policy[s, best_action] = 1.0
    
    return policy, V

In [13]:
def run_policy(env, policy):
    state = env.reset()
    curr = 0
    trajectory = ()
    action = 0
    next_state, reward, done, info = env.step(action)
    env.render()
    print(action)
    state=next_state
    for i in range(len(env.seq) - 2):
        direction = np.where(policy[curr] == 1.0)[0][0]
        action = (3 * action + direction) % 4
        next_state, reward, done, info = env.step(action)
        env.render()
        state=next_state
        trajectory += (direction,)
        curr = env.states_dic[trajectory]
        if done:
            print("Reward: {} | Actions: {}".format(reward, info['actions']))
            break

In [23]:
env = Lattice2DLinearEnv("HHPHPHHPPH")
policy, v = value_iteration(env)

C:\Users\binyu\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Your agent was trapped! Ending the episode.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [24]:
v

array([4., 4., 3., ..., 0., 0., 0.])

In [25]:
policy

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [26]:
run_policy(env, policy)

  (Left)
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
*********HH*********
********************
********************
********************
********************
********************
********************
********************
********************
********************
0
  (Down)
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
*********HH*********
*********P**********
********************
********************
********************
********************
********************
********************
********************
********************
  (Right)
********************
********************
********************
********************
********************
********************
****